In [1]:
import os
import glob
import numpy as np
import librosa
import soundfile as sf

# Create directory for saving preprocessed audio files
os.makedirs('preprocessed_call_snippets', exist_ok=True)

def preprocess_and_save_audio(filepath):
    # Extract filename without path
    filename = os.path.basename(filepath)
    # Extract ebird_code and indices from the filename
    ebird_code, idx, event_idx = filename.replace('.wav', '').split('_')
    
    # Load the audio file using librosa
    y, sr = librosa.load(filepath, sr=None)
    
    # Convert to mono if it's stereo
    y_mono = librosa.to_mono(y)
    
    # Resample the audio to exactly 22,000 Hz
    target_sr = 22000
    y_resampled = librosa.resample(y_mono, orig_sr=sr, target_sr=target_sr)
    
    # Calculate the number of samples for 5 seconds
    target_samples = target_sr * 5

    # Trim or pad the audio to exactly 5 seconds
    y_trimmed = librosa.util.fix_length(y_resampled, size=target_samples)
    
    # Save the processed audio to a WAV file
    output_filename = f"{ebird_code}_{idx}_{event_idx}.wav"
    output_filepath = os.path.join('preprocessed_call_snippets', output_filename)
    sf.write(output_filepath, y_trimmed, samplerate=target_sr)
    
    # print(f"Processed and saved: {output_filename}")
    return output_filename

# Get list of audio files in 'bird_snippets'
audio_files = glob.glob('bird_snippets/*.wav')

# Process each audio file
for filepath in audio_files:
    preprocess_and_save_audio(filepath)

print("Preprocessing complete. Preprocessed audio files are saved in 'preprocessed_call_snippets/' directory.")


Preprocessing complete. Preprocessed audio files are saved in 'preprocessed_call_snippets/' directory.


In [2]:
# to save metadata for the whole preprocessed files:
import csv

# Open a CSV file to write metadata
with open('preprocessed_call_snippets/metadata.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write header
    csvwriter.writerow(['filename', 'ebird_code', 'idx', 'event_idx'])
    
    # Process each audio file
    for filepath in audio_files:
        filename = preprocess_and_save_audio(filepath)
        # Write metadata
        ebird_code, idx, event_idx = filename.replace('.wav', '').split('_')
        csvwriter.writerow([filename, ebird_code, idx, event_idx])
